## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [3]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'
sheet_value = 'TOP VALOR'
sheet_packs = 'TOP PACKS'
sheet_services = 'TOP SERVICIOS'

# Range to read (change this for each project)
start_row = 8
end_row = 150
usecols = 'A:X'

print("Variables defined correctly!")

Variables defined correctly!


## Verify Columns in 'Active' Sheet.

Verify the number of columns in the "Active" sheet to ensure the range is within bounds. We will use the range in all three sheets

In [ ]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

## Extract Data from all six Sheets

Extract data from all three Sheets within the specified range and convert it directly to a DataFrame.

In [5]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)
value_df = load_sheet_as_dataframe(excel_file_path, 'TOP VALOR', start_row, end_row, usecols)
packs_df = load_sheet_as_dataframe(excel_file_path, 'TOP PACKS', start_row, end_row, usecols)
services_df = load_sheet_as_dataframe(excel_file_path, 'TOP SERVICIOS', start_row, end_row, usecols)

# Display the DataFrames
print("Active DataFrame:")
display(active_df.head(99))

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

print("value DataFrame:")
display(value_df.head())

print("Packs DataFrame:")
display(packs_df.head())

print("Services DataFrame:")
display(services_df.head())


Data from 'ACTIVOS' sheet loaded successfully.
Data from 'CANCELADOS' sheet loaded successfully.
Data from 'PTE FIRMA' sheet loaded successfully.
Data from 'TOP VALOR' sheet loaded successfully.
Data from 'TOP PACKS' sheet loaded successfully.
Data from 'TOP SERVICIOS' sheet loaded successfully.
Active DataFrame:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,NaN,NaN,7.0,16.0,15.0,22.0,169
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,6
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,NaN,NaN,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,NaN,NaN,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,NaN,NaN,23.0,15.0,24.0,13.0,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Fibra 600 con permanencia.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
94,victor.justamante@finetwork.com,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4
95,yicel.patricia@originaltelecom.es,NaN,NaN,16.0,7.0,9.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,22.0,18.0,11.0,NaN,NaN,NaN,121
96,yzabelly.gomes@originaltelecom.es,NaN,NaN,14.0,15.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.0,13.0,NaN,NaN,NaN,NaN,81


Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,111.1,56,88,54.1,50,111.2,70,49,24,1971
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,2.0,NaN,NaN,NaN,3.0,2.0,5.0,3.0,10.0,...,NaN,NaN,10.0,NaN,NaN,NaN,4.0,4.0,3.0,57
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,albertocanto@originaltelecom.es,1.0,5.0,NaN,NaN,NaN,5.0,5.0,3.0,4.0,...,1.0,NaN,4.0,NaN,NaN,4.0,3.0,NaN,NaN,42
4,albertosanchez@originaltelecom.es,3.0,1.0,NaN,NaN,3.0,3.0,6.0,NaN,4.0,...,3.0,2.0,10.0,NaN,NaN,3.0,NaN,NaN,NaN,47


Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,218,189,104,64,261,241,224,193,204,...,200,123,222,115,65,325,302,349,357,4395
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,3.0,4.0,NaN,NaN,1.0,15.0,2.0,4.0,6.0,...,5.0,5.0,15.0,NaN,NaN,7.0,13.0,10.0,16.0,113
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4
3,albertocanto@originaltelecom.es,4.0,7.0,NaN,NaN,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,14.0,NaN,NaN,10.0,10.0,11.0,9.0,128
4,albertosanchez@originaltelecom.es,11.0,6.0,NaN,NaN,6.0,15.0,17.0,12.0,6.0,...,6.0,2.0,10.0,NaN,NaN,20.0,7.0,17.0,10.0,169


Packs DataFrame:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,266,159,263.1,118,78,436,403,464,404,5347
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,6.0,5.0,NaN,NaN,5.0,18.0,3.0,3.0,11.0,...,3.0,3.0,13.0,NaN,NaN,3.0,12.0,12.0,17.0,128
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,4
3,albertocanto@originaltelecom.es,7.0,6.0,NaN,NaN,6.0,3.0,15.0,10.0,11.0,...,21.0,3.0,18.0,NaN,NaN,17.0,21.0,13.0,10.0,175
4,albertosanchez@originaltelecom.es,12.0,8.0,NaN,NaN,5.0,7.0,25.0,21.0,9.0,...,8.0,8.0,10.0,NaN,NaN,19.0,12.0,22.0,10.0,197


Services DataFrame:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,NaN,NaN,7.0,16.0,15.0,22.0,169
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,6
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,NaN,NaN,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,NaN,NaN,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,NaN,NaN,23.0,15.0,24.0,13.0,254


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [6]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)
value_df = replace_nan_with_zero(value_df)
packs_df = replace_nan_with_zero(packs_df)
services_df = replace_nan_with_zero(services_df)



# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head(88))

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

print("Value DataFrame after replacing NaN:")
display(value_df.head())

print("Packs DataFrame after replacing NaN:")
display(packs_df.head())

print("Services DataFrame after replacing NaN:")
display(services_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,6
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,paulavilla@originaltelecom.es,0.0,0.0,14.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,16.0,3.0,0.0,0.0,0.0,0.0,57
84,pilar.deval@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,7.0,...,17.0,10.0,7.0,0.0,0.0,19.0,9.0,18.0,16.0,137
85,rosalba.borgesgonzalez@cgi.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2
86,[PACK] Fibra 600 Mbps y 10 GB.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2


Canceled DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,111.1,56,88,54.1,50,111.2,70,49,24,1971
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,2.0,5.0,3.0,10.0,...,0.0,0.0,10.0,0.0,0.0,0.0,4.0,4.0,3.0,57
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,1.0,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,42
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,4.0,...,3.0,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,47


Pending Signature DataFrame after replacing NaN:


...

,Inbound Telec.Orig.Sevilla,218,189,104,64,261,241,224,193,204,...,200,123,222,115,65,325,302,349,357,4395
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,5.0,5.0,15.0,0.0,0.0,7.0,13.0,10.0,16.0,113
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,14.0,0.0,0.0,10.0,10.0,11.0,9.0,128
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,6.0,...,6.0,2.0,10.0,0.0,0.0,20.0,7.0,17.0,10.0,169


Packs DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,266,159,263.1,118,78,436,403,464,404,5347
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,3.0,13.0,0.0,0.0,3.0,12.0,12.0,17.0,128
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,4
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,21.0,3.0,18.0,0.0,0.0,17.0,21.0,13.0,10.0,175
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,8.0,10.0,0.0,0.0,19.0,12.0,22.0,10.0,197


Services DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,6
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254


retention.ipynb

In [7]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [8]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)
value_df = ensure_all_agents(value_df, agents_list)
packs_df = ensure_all_agents(packs_df, agents_list)
services_df = ensure_all_agents(services_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head(88))

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

print("Value DataFrame after ensuring all agents:")
display(value_df.head())

print("Packs DataFrame after ensuring all agents:")
display(packs_df.head())

print("Services DataFrame after ensuring all agents:")
display(services_df.head())

Added 2 missing agents and removed 55 agents not in the list.
Added 2 missing agents and removed 35 agents not in the list.
Added 12 missing agents and removed -7 agents not in the list.
Added 2 missing agents and removed 30 agents not in the list.
Added 2 missing agents and removed 52 agents not in the list.
Added 2 missing agents and removed 55 agents not in the list.
Active DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,9.0,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170
11,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,32.0,9.0,0.0,0.0,0.0,0.0,68
12,beatriz.gomez@originaltelecom.es,6.0,9.0,0.0,0.0,5.0,6.0,6.0,8.0,15.0,...,10.0,4.0,7.0,0.0,0.0,8.0,13.0,22.0,15.0,162
14,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,11.0,16.0,10.0,15.0,14.0,...,17.0,7.0,13.0,0.0,0.0,19.0,11.0,21.0,13.0,216
16,cesar.arnaldo@originaltelecom.es,0.0,0.0,16.0,7.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,15.0,9.0,16.0,0.0,0.0,0.0,98
21,david.molero@originaltelecom.es,8.0,5.0,0.0,0.0,17.0,15.0,25.0,19.0,24.0,...,16.0,6.0,0.0,0.0,0.0,24.0,15.0,23.0,11.0,254


Canceled DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,111.1,56,88,54.1,50,111.2,70,49,24,1971
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,2.0,5.0,3.0,10.0,...,0.0,0.0,10.0,0.0,0.0,0.0,4.0,4.0,3.0,57
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,1.0,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,42
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,4.0,...,3.0,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,47
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,56
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,4.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,50


Pending Signature DataFrame after ensuring all agents:


...

,Inbound Telec.Orig.Sevilla,218,189,104,64,261,241,224,193,204,...,200,123,222,115,65,325,302,349,357,4395
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,5.0,5.0,15.0,0.0,0.0,7.0,13.0,10.0,16.0,113
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,14.0,0.0,0.0,10.0,10.0,11.0,9.0,128
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,6.0,...,6.0,2.0,10.0,0.0,0.0,20.0,7.0,17.0,10.0,169
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,9.0,11.0,0.0,98
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,4.0,5.0,0.0,0.0,0.0,12.0,8.0,11.0,6.0,108


Packs DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,266,159,263.1,118,78,436,403,464,404,5347
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,3.0,13.0,0.0,0.0,3.0,12.0,12.0,17.0,128
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,21.0,3.0,18.0,0.0,0.0,17.0,21.0,13.0,10.0,175
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,8.0,10.0,0.0,0.0,19.0,12.0,22.0,10.0,197
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,15.0,24.0,0.0,125
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,8.0,...,8.0,11.0,0.0,0.0,0.0,12.0,10.0,12.0,13.0,130


Services DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,9.0,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [9]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)
value_df = add_email_values(value_df, email_values)
packs_df = add_email_values(packs_df, email_values)
services_df = add_email_values(services_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

print("Value DataFrame with email values:")
display(value_df.head())

print("Packs DataFrame with email values:")
display(packs_df.head())

print("Services DataFrame with email values:")
display(services_df.head())

Active DataFrame with email values:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,197,336,171,98,526,475,556,508,6659,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170,5


Canceled DataFrame with email values:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,56,88,54.1,50,111.2,70,49,24,1971,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,2.0,5.0,3.0,10.0,...,0.0,10.0,0.0,0.0,0.0,4.0,4.0,3.0,57,1
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,42,2
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,4.0,...,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,47,3
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,56,4
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,50,5


Pending Signature DataFrame with email values:


...

,Inbound Telec.Orig.Sevilla,218,189,104,64,261,241,224,193,204,...,123,222,115,65,325,302,349,357,4395,email_value
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,5.0,15.0,0.0,0.0,7.0,13.0,10.0,16.0,113,1
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,4.0,14.0,0.0,0.0,10.0,10.0,11.0,9.0,128,2
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,6.0,...,2.0,10.0,0.0,0.0,20.0,7.0,17.0,10.0,169,3
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,9.0,11.0,0.0,98,4
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,5.0,0.0,0.0,0.0,12.0,8.0,11.0,6.0,108,5


Packs DataFrame with email values:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,159,263.1,118,78,436,403,464,404,5347,email_value
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,13.0,0.0,0.0,3.0,12.0,12.0,17.0,128,1
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,3.0,18.0,0.0,0.0,17.0,21.0,13.0,10.0,175,2
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,10.0,0.0,0.0,19.0,12.0,22.0,10.0,197,3
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,15.0,24.0,0.0,125,4
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,8.0,...,11.0,0.0,0.0,0.0,12.0,10.0,12.0,13.0,130,5


Services DataFrame with email values:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,197,336,171,98,526,475,556,508,6659,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170,5


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [10]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)
value_df = sort_by_email_value(value_df)
packs_df = sort_by_email_value(packs_df)
services_df = sort_by_email_value(services_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head()) 

print("Sorted Canceled DataFrame:")
display(canceled_df.head())

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())

print("Sorted Canceled DataFrame:")
display(value_df.head())

print("Sorted Pending Signature DataFrame:")
display(packs_df.head())

print("Sorted Pending Signature DataFrame:")
display(services_df.head())

Sorted Active DataFrame:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,197,336,171,98,526,475,556,508,6659,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170,5


Sorted Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,56,88,54.1,50,111.2,70,49,24,1971,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,2.0,5.0,3.0,10.0,...,0.0,10.0,0.0,0.0,0.0,4.0,4.0,3.0,57,1
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,42,2
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,4.0,...,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,47,3
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,56,4
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,50,5


Sorted Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,218,189,104,64,261,241,224,193,204,...,123,222,115,65,325,302,349,357,4395,email_value
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,5.0,15.0,0.0,0.0,7.0,13.0,10.0,16.0,113,1
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,4.0,14.0,0.0,0.0,10.0,10.0,11.0,9.0,128,2
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,6.0,...,2.0,10.0,0.0,0.0,20.0,7.0,17.0,10.0,169,3
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,9.0,11.0,0.0,98,4
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,5.0,0.0,0.0,0.0,12.0,8.0,11.0,6.0,108,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,159,263.1,118,78,436,403,464,404,5347,email_value
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,13.0,0.0,0.0,3.0,12.0,12.0,17.0,128,1
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,3.0,18.0,0.0,0.0,17.0,21.0,13.0,10.0,175,2
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,10.0,0.0,0.0,19.0,12.0,22.0,10.0,197,3
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,15.0,24.0,0.0,125,4
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,8.0,...,11.0,0.0,0.0,0.0,12.0,10.0,12.0,13.0,130,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,197,336,171,98,526,475,556,508,6659,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170,5


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [11]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)
value_df = remove_email_value_column(value_df)
packs_df = remove_email_value_column(packs_df)
services_df = remove_email_value_column(services_df)


# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

print("Value DataFrame after removing 'email_value' column:")
display(value_df.head())

print("Packs DataFrame after removing 'email_value' column:")
display(packs_df.head())

print("Services DataFrame after removing 'email_value' column:")
display(services_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

Active DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,9.0,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170


Canceled DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,111,114,60,54,128,108,135,112,114.1,...,111.1,56,88,54.1,50,111.2,70,49,24,1971
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,2.0,5.0,3.0,10.0,...,0.0,0.0,10.0,0.0,0.0,0.0,4.0,4.0,3.0,57
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,1.0,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,42
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,4.0,...,3.0,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,47
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,56
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,4.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,50


Pending Signature DataFrame after removing 'email_value' column:


...

Packs DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,247,224,124,93,275,278,282,236,263,...,266,159,263.1,118,78,436,403,464,404,5347
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,3.0,13.0,0.0,0.0,3.0,12.0,12.0,17.0,128
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,21.0,3.0,18.0,0.0,0.0,17.0,21.0,13.0,10.0,175
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,8.0,10.0,0.0,0.0,19.0,12.0,22.0,10.0,197
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,15.0,24.0,0.0,125
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,8.0,...,8.0,11.0,0.0,0.0,0.0,12.0,10.0,12.0,13.0,130


Services DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,325,271,168,106,350,351,341,288,311,...,340,197,336,171,98,526,475,556,508,6659
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,6.0,5.0,17.0,0.0,0.0,7.0,16.0,15.0,22.0,169
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,21.0,5.0,21.0,0.0,0.0,20.0,22.0,15.0,12.0,209
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,10.0,...,12.0,8.0,13.0,0.0,0.0,23.0,15.0,24.0,13.0,254
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,18.0,26.0,0.0,159
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,10.0,...,9.0,12.0,0.0,0.0,0.0,19.0,12.0,15.0,16.0,170


Values from the second column of active_df:
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroy

## Process Data from all DataFrames

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [12]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2
value_sums = value_df.iloc[:, 1:].sum(axis=1) / 2
packs_sums = packs_df.iloc[:, 1:].sum(axis=1) / 2
services_sums = services_df.iloc[:, 1:].sum(axis=1) / 2

print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head(66))# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())
print("Calculated sums for 'Value' DataFrame:")
print(value_sums.head())
print("Calculated sums for 'Packs' DataFrame:")
print(packs_sums.head())
print("Calculated sums for 'Services' DataFrame:")
print(services_df.head())

Calculated sums for 'Active' DataFrame:
1     169.0
3     209.0
4     254.0
8     159.0
10    170.0
11     68.0
12    162.0
65    116.0
14    216.0
16     98.0
21    254.0
25    223.0
29    134.0
38    223.0
40    257.0
41    185.0
43    104.0
46     53.0
52    221.0
51     86.0
54    169.0
24    130.0
60    146.0
98      0.0
99      0.0
64    144.0
63    195.0
62    111.0
69    114.0
70    137.0
72    238.0
73    158.0
61     76.0
77    156.0
78    122.0
79    107.0
80    182.0
83     57.0
84    137.0
90     96.0
91    137.0
95    121.0
96     81.0
dtype: float64
Calculated sums for 'Canceled' DataFrame:
1    57.0
3    42.0
4    47.0
5    56.0
6    50.0
dtype: float64
Calculated sums for 'Pending Signature' DataFrame:
0    6.0
1    4.0
2    3.0
3    1.0
4    2.0
dtype: float64
Calculated sums for 'Value' DataFrame:
1    113.0
3    128.0
4    169.0
5     98.0
6    108.0
dtype: float64
Calculated sums for 'Packs' DataFrame:
1    128.0
3    175.0
4    197.0
5    125.0
6    130.0
dtype: f

## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from all five DataFrame

### Update finalFile Excel sheet with Active Data

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame in column J.

In [13]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'.


### Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column K.

In [14]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Canceled data.


### Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column L.

In [15]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Pending Signature data.


### Update finalFile Excel Sheet with Values Data

Update the "finalFile" Excel sheet with the calculated values from the Values DataFrame in column M.

In [16]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 13  # Column M
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Values data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, value_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Values data.


### Update finalFile Excel Sheet with Packs Data

Update the "finalFile" Excel sheet with the calculated values from the Packs DataFrame in column N.

In [17]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 14  # Column N
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Packs data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, packs_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Packs data.


### Update finalFile Excel Sheet with Services Data

Update the "finalFile" Excel sheet with the calculated values from the Services DataFrame in column O.

In [18]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 15  # Column O
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Services data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, services_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Services data.
